In [ ]:
!pip install langchain langchain-community
!pip install faiss-cpu #facebook ai similarity search
!pip install pypdf python-docx
!pip install sentence-transformers
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded=files.upload()

file_path=list(uploaded.keys())[0]
print("Uploaded:",file_path)

Saving lang.txt to lang (1).txt
Uploaded: lang (1).txt


In [ ]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

if file_path.endswith('.pdf'):
  loader=PyPDFLoader(file_path)
elif file_path.endswith('.docx'):
  loader=Docx2txtLoader(file_path)
else:
  loader=TextLoader(file_path)

docs=loader.load()

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=splitter.split_documents(docs)

print(f"Total Chunks:{len(documents)}")

Total Chunks:1


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore=FAISS.from_documents(documents,embeddings)

/tmp/ipython-input-909571181.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

flan_pipeline=pipeline(
    "text2text-generation",
     model="google/flan-t5-base",
     max_length=512
    )

llm=HuggingFacePipeline(pipeline=flan_pipeline)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-582235684.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=flan_pipeline)


In [ ]:
from langchain.chains import RetrievalQA

qa=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k":3}),
    chain_type="stuff"
)

query="Give me a short summary of the document"
result=qa.run(query)
print(result)

Technology is evolving faster than ever before, shaping the way we work, communicate, and live our daily lives. From artificial intelligence to renewable energy, innovations are not only transforming industries but also redefining human potential. In this rapidly changing world, adaptability and continuous learning are the keys to thriving.


In [ ]:
while True:
  q=input("Ask a question(or 'exit'):")
  if q.lower()=='exit':
    break
  print("Answer:",qa.run(q))

Answer: shaping the way we work, communicate, and live our daily lives
Answer: adaptability and continuous learning
Answer: Artificial intelligence to renewable energy
Answer: Hello
Answer: shaping the way we work, communicate, and live our daily lives
Answer: Artificial intelligence to renewable energy, innovations are not only transforming industries but also redefining human potential.
Answer: if you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer: no
Answer: not enough information
Answer: not enough information
Answer: not enough information
Answer: Good.
Answer: dance is a type of dance.
Answer: not enough information
Answer: is an American singer
